In [2]:
# ----------------------------
# 📦 Imports and Setup
# ----------------------------
import pandas as pd
import datetime
from dash import Dash, html, dcc, Output, Input
import dash_leaflet as dl
import dash_leaflet.express as dlx


In [3]:

# ----------------------------
# 📊 Load and Prepare Data
# ----------------------------
df = pd.read_excel("cw_perform_event_df.xlsx")
df = df[df["vehicleType"].isin(["FL", "ARC"])]  # Filter only FL and ARC

df['start'] = pd.to_datetime(df['start'])
df['end'] = pd.to_datetime(df['end'])
df['date'] = df['start'].dt.date

# Clean invalid locations
df = df.dropna(subset=['completionLatitude', 'completionLongitude'])


In [4]:
# ----------------------------
# 🚀 Dash App Setup
# ----------------------------
app = Dash(__name__)
server = app.server

app.layout = html.Div([
    html.H2("Truck Tour Viewer with Map"),
    html.Div([
        html.Label("Select Date:"),
        dcc.Dropdown(
            id="date-dropdown",
            options=[{"label": str(date), "value": str(date)} for date in sorted(df['date'].unique())],
            value=str(sorted(df['date'].unique())[0])
        ),
        html.Label("Select Truck ID:"),
        dcc.Dropdown(id="truck-dropdown")
    ], style={"width": "30%", "display": "inline-block", "verticalAlign": "top"}),

    html.Div([
        dl.Map(
            id="map",
            center=(48.7656, 11.4237),
            zoom=11,
            style={"width": "100%", "height": "700px"},
            children=[dl.TileLayer()]
        )
    ], style={"width": "68%", "display": "inline-block", "paddingLeft": "2%"})
])

In [5]:
# ----------------------------
# 🔁 Populate Truck Dropdown Based on Selected Date
# ----------------------------
@app.callback(
    Output("truck-dropdown", "options"),
    [Input("date-dropdown", "value")]
)
def update_truck_dropdown(selected_date):
    date_obj = datetime.datetime.strptime(selected_date, "%Y-%m-%d").date()
    trucks = df[df['date'] == date_obj]['truck'].unique()
    return [{"label": truck, "value": truck} for truck in sorted(trucks)]

In [6]:
# ----------------------------
# 🗺️ Map Rendering with Color Coded Stops and Mismatch Detection
# ----------------------------
@app.callback(
    Output("map", "children"),
    [Input("date-dropdown", "value"), Input("truck-dropdown", "value")]
)
def update_map(selected_date, truck_id):
    if not selected_date or not truck_id:
        return [dl.TileLayer()]

    df_filtered = df[(df["date"] == datetime.datetime.strptime(selected_date, "%Y-%m-%d").date()) &
                     (df["truck"] == truck_id)].sort_values("start")

    if df_filtered.empty:
        return [dl.TileLayer()]

    coordinates = list(zip(df_filtered["completionLatitude"], df_filtered["completionLongitude"]))

    first_coord = coordinates[0]
    last_coord = coordinates[-1]

    same_start_end = (abs(first_coord[0] - last_coord[0]) < 0.0001 and abs(first_coord[1] - last_coord[1]) < 0.0001)
    start_color = "green" if same_start_end else "orange"

    # Draw black polyline regardless of truck
    polyline = dl.Polyline(positions=coordinates, color="black", weight=3)

    # Markers with appropriate color
    markers = []
    for i, row in enumerate(df_filtered.itertuples()):
        lat, lon = row.completionLatitude, row.completionLongitude
        if i == 0:
            color = start_color
        else:
            color = "blue"

        popup = dl.Popup([
            html.B(f"Truck: {row.truck}"), html.Br(),
            f"Client: {row.clientAddress}", html.Br(),
            f"Start: {row.start}", html.Br(),
            f"End: {row.end}", html.Br(),
            f"CO₂ Emission: {getattr(row, 'event_co2Emission', 'N/A')}", html.Br(),
            f"Fuel Rate: {getattr(row, 'event_fuel_rate', 'N/A')}", html.Br(),
            f"Speed: {getattr(row, 'wheel_speed', 'N/A')}"
        ], maxWidth=300)

        markers.append(
            dl.CircleMarker(
                center=(lat, lon),
                radius=6,
                color=color,
                fill=True,
                fillOpacity=0.9,
                children=[popup]
            )
        )

    # Legend
    legend = dlx.categorical_colorbar(
        categories=["Start/End (OK)", "Start/End (Mismatch)", "Client Stop"],
        colorscale=["green", "orange", "blue"],  # ✅ Added this line
        width=350,
        height=30,
        position="bottomright"
)


    return [dl.TileLayer(), polyline, *markers, legend]


In [7]:

# ----------------------------
# 🚀 Run the App
# ----------------------------
if __name__ == '__main__':
    app.run_server(debug=True)
